<a href="https://colab.research.google.com/github/AgungA1/OCR/blob/main/Char_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from tqdm import tqdm
import cv2
import random
import numpy as np
import pickle
from google.colab.patches import cv2_imshow

# Direktori data training
DATADIR = "/content/drive/MyDrive/Colab Notebooks/Computer Vision/Tugas Akhir OCR/dataset/training/"
dirs = []
training_data = []
width, height = 100, 100

# Looping direktori data training untuk diambil nama karakternya
for char_name in sorted(os.listdir(DATADIR)):
  dirs.append(char_name)

# Looping semua image data training untuk diubah menjadi array
for char_name in dirs:
  path = os.path.join(DATADIR, char_name)
  class_number = dirs.index(char_name)
  for img in tqdm(os.listdir(path)):
    try:
      img_array = cv2.imread((os.path.join(path, img)), cv2.IMREAD_ANYCOLOR)
      # gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
      new_array = cv2.resize(img_array, (width, height))
      training_data.append([new_array, class_number])
    except Exception as e:
      print(e)

random.shuffle(training_data)
X = []
Y = []

for feature, label in training_data:
  X.append(feature)
  Y.append(label)

X = np.array(X).reshape(-1, width, height, 3)

# Tulis ke file pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("Y.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

100%|██████████| 63/63 [00:12<00:00,  5.22it/s]


In [3]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

# Load file pickle
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in = open("Y.pickle", "rb")
Y = pickle.load(pickle_in)
Y = to_categorical(Y)
X = X / 255.0
width, height = 100, 100

# Input layer
inputs = Input(shape=(width, height, 3))
conv_layer = ZeroPadding2D(padding=(2, 2))(inputs)
conv_layer = Conv2D(16, (5, 5), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(64, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
flaten = Flatten()(conv_layer)
fc_layer = Dense(256, activation='relu')(flaten)
fc_layer = Dense(64, activation='relu')(fc_layer)

# Output layer
outputs = Dense(35, activation='softmax')(fc_layer)
adam = Adam(learning_rate=0.0001)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, Y, epochs=20, verbose=2)
score = model.evaluate(X, Y)
print(f"Akurasi model: {score[1] * 100:.2f}%")

Epoch 1/20
43/43 - 7s - loss: 3.1905 - accuracy: 0.1395 - 7s/epoch - 169ms/step
Epoch 2/20
43/43 - 1s - loss: 2.2690 - accuracy: 0.4032 - 501ms/epoch - 12ms/step
Epoch 3/20
43/43 - 1s - loss: 1.6072 - accuracy: 0.6092 - 535ms/epoch - 12ms/step
Epoch 4/20
43/43 - 1s - loss: 1.2083 - accuracy: 0.7202 - 515ms/epoch - 12ms/step
Epoch 5/20
43/43 - 0s - loss: 0.9315 - accuracy: 0.7684 - 454ms/epoch - 11ms/step
Epoch 6/20
43/43 - 0s - loss: 0.7279 - accuracy: 0.8159 - 444ms/epoch - 10ms/step
Epoch 7/20
43/43 - 0s - loss: 0.5365 - accuracy: 0.8605 - 447ms/epoch - 10ms/step
Epoch 8/20
43/43 - 0s - loss: 0.3953 - accuracy: 0.8963 - 450ms/epoch - 10ms/step
Epoch 9/20
43/43 - 0s - loss: 0.2960 - accuracy: 0.9211 - 441ms/epoch - 10ms/step
Epoch 10/20
43/43 - 0s - loss: 0.2100 - accuracy: 0.9394 - 448ms/epoch - 10ms/step
Epoch 11/20
43/43 - 0s - loss: 0.1410 - accuracy: 0.9627 - 446ms/epoch - 10ms/step
Epoch 12/20
43/43 - 0s - loss: 0.1077 - accuracy: 0.9766 - 449ms/epoch - 10ms/step
Epoch 13/20
43/

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 zero_padding2d (ZeroPaddin  (None, 104, 104, 3)       0         
 g2D)                                                            
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 16)      1216      
                                                                 
 max_pooling2d (MaxPooling2  (None, 50, 50, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 32)        9248  

In [6]:
model.save('/content/drive/MyDrive/Colab Notebooks/Computer Vision/Tugas Akhir OCR/charv2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
